In [ ]:
# 合并文件
import pandas as pd
import os
import glob

# 文件夹路径
folder_path = "/Users/jackli/Desktop/python/transformer/processed_data"

# 获取该文件夹下所有 csv 文件路径
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

all_columns = set()  # 收集所有列名
valid_dfs = []       # 存放处理后的DataFrame

# 第一次遍历：收集所有列名
for file in csv_files:
    try:
        df = pd.read_csv(file)
        all_columns.update(df.columns)
    except Exception as e:
        print(f"❌ 读取文件失败 {file}: {e}")

# 转成固定顺序的列表
all_columns = list(all_columns)

# 第二次遍历：提取最大连续非缺失区间
for file in csv_files:
    try:
        df = pd.read_csv(file)
        column = 'flow'

        # 跳过不含flow列或该列全为NaN的文件
        if column not in df.columns or df[column].dropna().empty:
            print(f"⚠️ 跳过文件（无有效flow数据）: {os.path.basename(file)}")
            continue

        # 标识连续非缺失区间
        mask = df[column].notna()
        df['group'] = (mask != mask.shift()).cumsum()

        # 找出最大非缺失区间
        non_missing_groups = df[mask].groupby('group').size()
        max_group = non_missing_groups.idxmax()

        # 保留最大连续区间
        df_cleaned = df[df['group'] == max_group].drop(columns='group').reset_index(drop=True)

        # 补齐缺失列
        for col in all_columns:
            if col not in df_cleaned.columns:
                df_cleaned[col] = pd.NA

        # 新增来源文件列
        df_cleaned['source_file'] = os.path.basename(file).split(".")[0]

        # 保持列顺序（source_file放最后）
        df_cleaned = df_cleaned[all_columns + ['source_file']]

        valid_dfs.append(df_cleaned)
        print(f"✅ 处理完成: {os.path.basename(file)}, 区间长度={len(df_cleaned)}")

    except Exception as e:
        print(f"❌ 处理文件出错 {file}: {e}")

# 合并所有处理后的文件
if valid_dfs:
    merged_df = pd.concat(valid_dfs, ignore_index=True)
    output_path = os.path.join(folder_path, "merged_max_nonmissing.csv")
    merged_df.to_csv(output_path, index=False)
    print(f"\n✅ 已保存合并文件: {output_path}")
else:
    print("⚠️ 未找到有效文件。")


✅ 处理完成: 2018-09-25.csv, 区间长度=112
✅ 处理完成: 2018-05-31.csv, 区间长度=215
✅ 处理完成: 2017-03-11.csv, 区间长度=95
✅ 处理完成: 2017-06-23.csv, 区间长度=105
✅ 处理完成: 2015-05-30.csv, 区间长度=131
✅ 处理完成: 2016-05-10.csv, 区间长度=160
✅ 处理完成: 2017-06-26.csv, 区间长度=82
✅ 处理完成: 2015-12-10.csv, 区间长度=126
✅ 处理完成: 2015-05-21.csv, 区间长度=76
✅ 处理完成: 2016-07-10.csv, 区间长度=67
✅ 处理完成: 2019-01-19.csv, 区间长度=235
✅ 处理完成: 2018-07-19.csv, 区间长度=272
✅ 处理完成: 2018-09-08.csv, 区间长度=221
✅ 处理完成: 2018-05-08.csv, 区间长度=74
✅ 处理完成: 2015-12-06.csv, 区间长度=102
✅ 处理完成: 2018-10-28.csv, 区间长度=80
✅ 处理完成: 2016-10-23.csv, 区间长度=79
✅ 处理完成: 2019-06-13.csv, 区间长度=81
✅ 处理完成: 2018-07-30.csv, 区间长度=60
✅ 处理完成: 2018-09-21.csv, 区间长度=108
✅ 处理完成: 2016-03-10.csv, 区间长度=118
✅ 处理完成: 2014-05-26.csv, 区间长度=134
✅ 处理完成: 2017-07-03.csv, 区间长度=125
✅ 处理完成: 2016-01-29.csv, 区间长度=100
✅ 处理完成: 2018-08-13.csv, 区间长度=66
✅ 处理完成: 2019-02-05.csv, 区间长度=78
✅ 处理完成: 2016-04-18.csv, 区间长度=172
✅ 处理完成: 2018-06-15.csv, 区间长度=141
✅ 处理完成: 2018-08-04.csv, 区间长度=55
✅ 处理完成: 2016-04-23.csv, 区间长度=102
✅ 处理完成: 2018-11-21.csv

/var/folders/bv/k15hzqs91t9crml70zw5t5qr0000gn/T/ipykernel_28689/2929487358.py:66: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged_df = pd.concat(valid_dfs, ignore_index=True)


In [ ]:
#标准化
import pandas as pd
import numpy as np
import os

# 文件路径
folder_path = "/Users/jackli/Desktop/python/transformer/processed_data"
input_path = os.path.join(folder_path, "merged_max_nonmissing.csv")
output_path = os.path.join(folder_path, "merged_max_nonmissing_scaled.csv")

# 读取文件
df = pd.read_csv(input_path)

# 检查列
if "flow" not in df.columns:
    raise ValueError("未在文件中找到 'flow' 列，请确认输入文件是否正确。")

# 复制一份避免修改原数据
df_scaled = df.copy()

# 对除 source_file 以外的所有列尝试做 log1p
numeric_cols = [col for col in df.columns if col != "source_file"]

for col in numeric_cols:
    # 非数值列会被跳过
    if pd.api.types.is_numeric_dtype(df[col]):
        df_scaled[col] = np.log1p(df[col])
    else:
        print(f"⚠️ 跳过非数值列: {col}")

# 对 flow 列进一步标准化
flow_log = df_scaled["flow"].copy()

# 只保留非零值（用于计算均值和方差）
flow_log_nonzero = flow_log[flow_log > 0]

mean = flow_log_nonzero.mean()
std = flow_log_nonzero.std()

# 对非零部分标准化
flow_scaled = (flow_log - mean) / std
df_scaled["flow"] = flow_scaled

# 输出结果
df_scaled.to_csv(output_path, index=False)
print(f"✅ 已保存标准化文件: {output_path}")
print(f"Flow 均值={mean:.4f}, 标准差={std:.4f}")


✅ 已保存标准化文件: /Users/jackli/Desktop/python/transformer/processed_data/merged_max_nonmissing_scaled.csv
Flow 均值=5.8329, 标准差=0.6035
